# Descripcion de datos

Se cuenta con una serie temporal de la demanda de electricidad (en megavatios, MW) para el estado de Victoria, Australia, que abarca desde el 31 de diciembre de 2011 hasta el 31 de diciembre de 2014. Estos datos se han obtenido del paquete `tsibbledata` de R. El conjunto de datos incluye 5 columnas y 52,608 registros completos. La descripción de cada columna es la siguiente:

- **Time**: Fecha y hora del registro.
- **Date**: Fecha del registro.
- **Demand**: Demanda de electricidad (MW).
- **Temperature**: Temperatura en Melbourne, la capital de Victoria.
- **Holiday**: Indicador de si el día es festivo (vacaciones).

# Limpieza de datos



Carga de Datos:
Se cargaron los datos desde un archivo CSV utilizando pd.read_csv.

Conversión de Fechas:
Se intentó convertir la columna Time a formato datetime con el argumento errors='coerce' para manejar errores como valores NaT (Not a Time).
Se identificaron y mostraron los valores que inicialmente no eran convertibles.

Corrección de Fechas Inválidas:
Se corrigieron las fechas no convertibles añadiendo un valor de tiempo (' 00:00:00') a aquellas fechas sin tiempo.
Se volvió a intentar la conversión de la columna Time a formato datetime.

Verificación y Limpieza:
Se verificó nuevamente si quedaban valores no convertibles y se aseguró que todas las fechas fueron correctamente convertidas.
Se eliminó la columna original Time y se renombró la columna convertida a Time.

Configuración del Índice:
Se estableció Time como el índice del DataFrame.
Se verificó y eliminaron duplicados en el índice.

Ajuste de Frecuencia y Ordenación:
Se ajustó la frecuencia del índice a intervalos de 30 minutos ('30min') y se ordenó el índice.

Guardado de Datos:
Se guardaron los datos limpiados y ajustados a un nuevo archivo CSV.

Carga y Verificación Posterior:
Se cargaron nuevamente los datos desde el archivo CSV ajustado.
Se verificó que Time era el índice.
Se realizó la conversión de tipos de datos para las columnas Demand, Temperature, y Holiday.

Verificación Final:
Se verificaron y se imprimieron los tipos de datos después de la conversión.

# Codigo de limpieza

In [ ]:
import pandas as pd

# Cargar los datos desde un archivo CSV
datos = pd.read_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\datos.csv")

# Intentar convertir la columna 'Time' a formato datetime con errores como NaT
datos['Time_converted'] = pd.to_datetime(datos['Time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Filtrar y mostrar los valores que no son convertibles (NaT)
invalid_dates = datos[datos['Time_converted'].isna()]
print("Datos no convertibles inicialmente:")
print(invalid_dates['Time'])

# Añadir el tiempo faltante a los valores no convertibles y volver a convertirlos
datos.loc[datos['Time_converted'].isna(), 'Time'] = datos['Time'] + ' 00:00:00'
datos['Time_converted'] = pd.to_datetime(datos['Time'], format="%Y-%m-%d %H:%M:%S")

# Comprobar que no hay datos que no se hayan convertido
invalid_dates = datos[datos['Time_converted'].isna()]
print("Datos no convertibles después de la corrección:")
print(invalid_dates['Time'])

# Eliminar la columna original 'Time' y renombrar 'Time_converted' a 'Time'
datos = datos.drop(columns=['Time'])
datos = datos.rename(columns={'Time_converted': 'Time'})

# Información inicial del DataFrame
print("Información inicial del DataFrame:")
print(datos.info())

# Número de filas iniciales
initial_rows = datos.shape[0]
print(f"Número de filas iniciales: {initial_rows}")

# Establecer 'Time' como el índice
datos = datos.set_index('Time')

# Verificar y contar duplicados en el índice
duplicated_times = datos.index.duplicated().sum()
print(f"Fechas o tiempos duplicados en el índice: {duplicated_times}")

# Eliminar duplicados en el índice si es necesario
if duplicated_times > 0:
    datos = datos[~datos.index.duplicated(keep='first')]

# Número de filas después de eliminar duplicados
rows_after_duplicates = datos.shape[0]
print(f"Número de filas después de eliminar duplicados: {rows_after_duplicates}")

# Ajustar la frecuencia y ordenar el índice
datos = datos.asfreq('30min')
datos = datos.sort_index()

# Información del DataFrame después de la reindexación
print("Información del DataFrame después de la reindexación:")
print(datos.info())

# Mostrar el DataFrame ajustado para referencia
print(datos.head(10))

# Guardar los datos limpiados y ajustados a un nuevo archivo CSV
datos.to_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\datos_limpiados.csv")

# Cargar los datos desde un archivo CSV
datos = pd.read_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\datos_limpiados.csv")
datos
datos = datos.set_index('Time')
# Verificar si 'Time' es el índice
if datos.index.name != 'Time':
    raise KeyError("El índice actual no es 'Time'.")
datos
datos.info()
# Conversión de tipos de datos
datos['Demand'] = datos['Demand'].astype(float)
datos['Temperature'] = datos['Temperature'].astype(float)
datos['Holiday'] = datos['Holiday'].astype(bool)

# Verificar los tipos de datos
print("Tipos de datos después de la conversión:")
print(datos.dtypes)